Run the cell below before beginning this lab!

In [ ]:
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import math
import statistics

def get_price_hist(ticker,period,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/marketdata/'+ticker+'/pricehistory'

    ##Define Payload
    payload = {'apikey': key,
    'periodType': 'year',
    'peirod':period,
    'frequencyType':'daily'}

    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()

    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        hist_data = pd.DataFrame(dictlist[0][1])
        hist_data['ticker'] = ticker
        hist_data['datetime'] = pd.to_datetime(hist_data['datetime'],unit='ms')
        return hist_data
    except:
        df = pd.DataFrame()
        return df
    

def get_fundamental_from_td(ticker,key):
    time.sleep(1)
    endpoint = 'https://api.tdameritrade.com/v1/instruments'
    projection = 'fundamental'

    ##Define Payload
    payload = {'apikey': key,
               'symbol' : ticker,
                'projection': projection,
                }
    
    ### make request
    try:
        content = requests.get(url = endpoint, params = payload)
    except:
        print('API error, please review.')
        
    ### Convert to dictionary
    dictlist = []
    data = content.json()
    for key, value in data.items():
        temp = [key,value]
        dictlist.append(temp)
        
    try:
        df = pd.DataFrame(dictlist[0][1]).T.reset_index(drop=True).iloc[0]
        return df
    except:
        print(dictlist)
        df = pd.DataFrame()
        print(ticker + " not valid.")
        return df

def scrub_fundamental_data(tickers,key):
    master = pd.DataFrame()
    count = 1
    for ticker in tickers:
        temp = get_fundamental_from_td(ticker,key)
        temp = pd.DataFrame(temp).T
        hist_data = get_annual_returns([ticker],key)
        try:
            temp.columns = ['beta', 'bookValuePerShare', 'currentRatio', 'divGrowthRate3Year',
           'dividendAmount', 'dividendDate', 'dividendPayAmount',
           'dividendPayDate', 'dividendYield', 'epsChange', 'epsChangePercentTTM',
           'epsChangeYear', 'epsTTM', 'grossMarginMRQ', 'grossMarginTTM', 'high52',
           'interestCoverage', 'low52', 'ltDebtToEquity', 'marketCap',
           'marketCapFloat', 'netProfitMarginMRQ', 'netProfitMarginTTM',
           'operatingMarginMRQ', 'operatingMarginTTM', 'pbRatio', 'pcfRatio',
           'peRatio', 'pegRatio', 'prRatio', 'quickRatio', 'returnOnAssets',
           'returnOnEquity', 'returnOnInvestment', 'revChangeIn', 'revChangeTTM',
           'revChangeYear', 'sharesOutstanding', 'shortIntDayToCover',
           'shortIntToFloat', 'ticker', 'totalDebtToCapital', 'totalDebtToEquity',
           'vol10DayAvg', 'vol1DayAvg', 'vol3MonthAvg']
            temp = pd.merge(temp, hist_data, on='ticker')
        except:
            continue
        
        master = master.append(temp).reset_index(drop=True)
        count+= 1
    return master

def scrub_price_hist(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        price_data = get_price_hist(ticker,1,key)
        master = master.append(price_data, ignore_index=True)
    return master

def get_annual_returns(tickers,key):
    master = pd.DataFrame()
    for ticker in tickers:
        try:
            price_data = get_price_hist(ticker,1,key).iloc[::-1]
            return_percent = price_data['close'].iloc[0] / price_data['close'].iloc[-1] -1
            df = pd.DataFrame({"ticker":ticker,  
                               "Annual_Return_Percent":return_percent},
                               index = [0])
            master = master.append(df, ignore_index=True)
        except:

            print('error with', ticker)
    return master

1. Pull the 1 year price history using the scrub_price_hist function of any stock you like.

2. Clean up the output so it mirrors our output for Apple.

3.  Think of another market proxy besides the S&P 500, pull it's data, and clean it up.

4. Merge the two data sets, so that it mirrors the 'consol_data' variable.  Then merge in the new proxy data and SPY data.

In [ ]:
# Run this cell first
spy_data = scrub_price_hist(['SPY'],key)
spy_data = spy_data[['close','datetime','ticker']]
spy_data = spy_data.sort_values(by = 'datetime', ascending = False)
spy_data['SPY_Return_%'] = spy_data['close'] / spy_data['close'].shift(-1) - 1
spy_data.rename(columns={'close': 'SPY_Close'}, inplace=True)
spy_data.drop(columns = ['ticker'], inplace = True)
spy_data.head()

5. Calculate the beta of your new stock relative to both SPY and your new market proxy.  Are they different significantly?
   Consider why they may be different.

6. Pick a stock we haven't looked at and grab its fundamental data.

7. Calculate the annual volatility of TQQQ.

8. Find the beta to SPY of AMD as well as its volatility and make a conclusion about its risk profile.

9. Get the fundamental data of AMC.  

10. Find the annualized volatility of an equally weighted portfolio of AMGN and PG.  Assume 252 trading days in a year and use one year of historicals.

11. Find the annualized volatility of AMGN and PG respecively with the same assumptions as question 10.

12. Find the volatility of XRX using data from June of 2021 only.  Compare it to its volatility in September of 2021.

13.  Find the 1-year value at risk for a portfolio of 25% FB and 75% MO.  Assume 252 trading days in a year, and use the most recent 1 year pricing data.

## -------------EXTEND YOUR UNDERSTANDING-------------

Assume your broker allows you to invest on margin with no cost of borrowing up to 30% (i.e. for each \\$1 you have, you can invest \\$1.30).  Assume you have $10,000, want to use all the margin, and are feeling very good about an equally weighted portfolio of AIV and AIRC.  What is your one-month VaR?  Assume 21 trading days in a month, and use the most recent year's worth of data to estimate volatility.  